In [1]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta
from SML.training_pipeline_new import model_dir
import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/tesla_stocks_fv_1_create_fv_td_10052024081806/executions
2024-05-10 10:19:22,487 WARNING: VersionWarning: Incremented version to `16`.

2024-05-10 10:19:23,632 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.5137 
2024-05-10 10:19:29,747 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/291253 elapsed<00:00 remaining<?

2024-05-10 10:19:38,395 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



Uploading: 0.000%|          | 0/45 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/561 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/693399/models/stock_pred_model/18


True

In [2]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
start_date = datetime.now() - timedelta(hours=72)
print(start_date.strftime("%Y-%m-%d"))

2024-05-07


In [4]:
end_date = datetime.now() - timedelta(hours=48)
print(end_date.strftime("%Y-%m-%d"))

2024-05-08


In [5]:
feature_view = fs.get_feature_view('tesla_stocks_fv', 3)
feature_view.init_batch_scoring(training_dataset_version=2)

In [6]:
print(feature_view.get_batch_query())

WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`ticker` `ticker`, `fg1`.`ticker` `join_pk_ticker`, `fg1`.`date` `join_evt_date`, `fg0`.`sentiment` `sentiment`, RANK() OVER (PARTITION BY `fg1`.`ticker`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `klittefr_featurestore`.`tesla_stock_3` `fg1`
INNER JOIN `klittefr_featurestore`.`news_sentiment_updated_3` `fg0` ON `fg1`.`ticker` = `fg0`.`ticker` AND `fg1`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`date` `date`, `right_fg0`.`ticker` `ticker`, `right_fg0`.`sentiment` `sentiment`
FROM right_fg0)


In [7]:
# we had problems fetching the data from fv with get_batch_data function, tried everything and it just doesnt work 
tesla_df_b = feature_view.get_batch_data(start_time=start_date, end_time=end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.20s) 


In [8]:
features_df = tesla_df_b.iloc[:,2]
features_df

0    0.010694
Name: sentiment, dtype: float64

In [9]:
tesla_df_b.head()

,date,ticker,sentiment
0,2024-05-08 00:00:00+00:00,TSLA,0.010694


In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
#OneHotEncoding the tesla_df_b column 'ticker'

tickers = tesla_df_b[['ticker']]

# Initializing OneHotEncoder
encoder = OneHotEncoder()

# Fitting and transforming the 'ticker' column
ticker_encoded_test = encoder.fit_transform(tickers)

# Converting the encoded column into a DataFrame
ticker_encoded_df_test = pd.DataFrame(ticker_encoded_test.toarray(), columns=encoder.get_feature_names_out(['ticker']))

# Concatenating the encoded DataFrame with the original DataFrame
tesla_df_b = pd.concat([tesla_df_b, ticker_encoded_df_test], axis=1)

# Dropping the original 'ticker' column
tesla_df_b.drop('ticker', axis=1, inplace=True)

In [12]:
# As X_train['date'] column exists and is in datetime format, we're converting it
tesla_df_b['year'] = tesla_df_b['date'].dt.year
tesla_df_b['month'] = tesla_df_b['date'].dt.month
tesla_df_b['day'] = tesla_df_b['date'].dt.day

# Dropping the original date column
tesla_df_b.drop(columns=['date'], inplace=True)

# Converting dataframe to numpy array
tesla_df_b_array = tesla_df_b.to_numpy()

# Reshaping the array to have a shape suitable for LSTM
tesla_df_b_array = np.expand_dims(tesla_df_b_array, axis=1)

2024-05-10 10:20:03,207 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [13]:
import joblib

the_model = mr.get_model("stock_pred_model", version=9)
model_dir = the_model.download()

model = joblib.load(model_dir + "/stock_prediction_model.pkl")

In [14]:
predictions = model.predict(tesla_df_b_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 582ms/step


In [15]:
predictions 

array([[0.18933737]], dtype=float32)

In [16]:
tesla_df_b['predictions'] = predictions.tolist()

In [17]:
# Assuming you have 'year', 'month', and 'day' columns in your DataFrame
tesla_df_b['date'] = pd.to_datetime(tesla_df_b[['year', 'month', 'day']])

# Now you can drop the 'year', 'month', and 'day' columns if you want
tesla_df_b.drop(columns=['year', 'month', 'day'], inplace=True)

In [18]:
tesla_df_b['date'] = pd.to_datetime(tesla_df_b['date'])

In [19]:
tesla_df_b

,sentiment,ticker_TSLA,predictions,date
0,0.010694,1.0,[0.1893373727798462],2024-05-08


In [27]:
# Convert the encoded DataFrame back to numpy array
ticker_encoded_array = ticker_encoded_df_test.to_numpy()

# Inverse transform the encoded array to retrieve the original values
original_tickers = encoder.inverse_transform(ticker_encoded_array)

# Convert the original_tickers array to a DataFrame
original_tickers_df = pd.DataFrame(original_tickers, columns=['ticker'])

# Concatenate the original ticker column with the remaining columns from tesla_df_b
tesla_df_b = pd.concat([tesla_df_b.drop(columns=['ticker_TSLA']), original_tickers_df], axis=1)


In [29]:
tesla_df_b.head()

,sentiment,predictions,date,ticker
0,0.010694,[0.1893373727798462],2024-05-08,TSLA


In [31]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.


In [38]:
results_fg = fs.get_or_create_feature_group(
    name= 'stock_prediction_results',
    version = 1,
    description = 'Predction of TSLA open stock price',
    primary_key = ['ticker'],
    event_time = ['date'],
    online_enabled = False,
)

2024-05-10 10:35:57,610 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [39]:
#Inserting the stock data into the stocks feature group
results_fg.insert(tesla_df_b, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/693399/fs/689222/fg/798071


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: stock_prediction_results_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/stock_prediction_results_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x21cf75e5f10>, None)